# Financial Modeling Prep (FMP) Setup Guide

**Provider:** Financial Modeling Prep (Fundamentals, ratios, DCF, SEC filings)

**Best for:** Financial statements, valuation metrics, DCF models, SEC filings

**Time:** 5 minutes

**Cost:** FREE tier (250 requests/day), Starter $14/mo, Premium $29/mo

---

**🎯 This is part of the STARTER TIER for the course!**

**Why FMP?** Best API for fundamental analysis — financial statements, ratios, DCF valuations, and SEC filings. Excellent for value investing and financial modeling.

## Step 1: Create FMP Account

1. Go to [site.financialmodelingprep.com](https://site.financialmodelingprep.com/)
2. Sign up

## Step 2: Get Your API Key

1. Go to [site.financialmodelingprep.com/developer/docs/dashboard](https://site.financialmodelingprep.com/developer/docs/dashboard)
2. Your API key is shown on the dashboard
3. **Copy the key**



## Step 3: Set Your API Key

Run this cell and paste your API key when prompted:

In [1]:
import os
from getpass import getpass

# Prompt for API key (hidden input)
api_key = getpass("Enter your FMP API key: ")
os.environ["FMP_API_KEY"] = api_key

print("✅ API key set!")
print(f"   Key format: {api_key[:8]}...{api_key[-4:]}")

Enter your FMP API key: ··········
✅ API key set!
   Key format: 48879f5c...204d


## Step 4: Install Required Packages

In [2]:
!pip install -q requests pandas

## Step 5: Test Your Connection - Company Profile

In [3]:
import requests

BASE_URL = "https://financialmodelingprep.com/api/v3"

def get_company_profile(ticker: str) -> dict:
    """Get company profile from FMP."""
    url = f"{BASE_URL}/profile/{ticker}"
    params = {"apikey": os.environ["FMP_API_KEY"]}
    response = requests.get(url, params=params)
    data = response.json()
    return data[0] if data else {}

# Test with Apple
profile = get_company_profile("AAPL")

if profile:
    print("✅ Connection successful!")
    print(f"   Company: {profile.get('companyName')}")
    print(f"   Price: ${profile.get('price')}")
    print(f"   Market Cap: ${profile.get('mktCap', 0)/1e9:.1f}B")
    print(f"   Industry: {profile.get('industry')}")
    print(f"   Sector: {profile.get('sector')}")
    print(f"   CEO: {profile.get('ceo')}")
else:
    print("❌ Connection failed - check your API key")

✅ Connection successful!
   Company: Apple Inc.
   Price: $248.71
   Market Cap: $3675.0B
   Industry: Consumer Electronics
   Sector: Technology
   CEO: Timothy D. Cook


## Step 6: Get Income Statement

In [4]:
import pandas as pd

def get_income_statement(ticker: str, period: str = "annual", limit: int = 5) -> pd.DataFrame:
    """Get income statement.

    Args:
        ticker: Stock symbol
        period: 'annual' or 'quarter'
        limit: Number of periods to retrieve
    """
    url = f"{BASE_URL}/income-statement/{ticker}"
    params = {
        "apikey": os.environ["FMP_API_KEY"],
        "period": period,
        "limit": limit
    }
    response = requests.get(url, params=params)
    return pd.DataFrame(response.json())

# Get Apple's income statement
income = get_income_statement("AAPL", "annual", 3)

print("✅ Income Statement retrieved!")
print(f"\n📊 Apple Income Statement ($ millions):")
key_cols = ["date", "revenue", "grossProfit", "operatingIncome", "netIncome"]
display_df = income[key_cols].copy()
for col in ["revenue", "grossProfit", "operatingIncome", "netIncome"]:
    display_df[col] = (display_df[col] / 1e6).round(0).astype(int)
print(display_df.to_string(index=False))

✅ Income Statement retrieved!

📊 Apple Income Statement ($ millions):
      date  revenue  grossProfit  operatingIncome  netIncome
2025-09-27   416161       195201           133050     112010
2024-09-28   391035       180683           123216      93736
2023-09-30   383285       169148           114301      96995


## Step 7: Get Balance Sheet

In [5]:
def get_balance_sheet(ticker: str, period: str = "annual", limit: int = 5) -> pd.DataFrame:
    """Get balance sheet."""
    url = f"{BASE_URL}/balance-sheet-statement/{ticker}"
    params = {
        "apikey": os.environ["FMP_API_KEY"],
        "period": period,
        "limit": limit
    }
    response = requests.get(url, params=params)
    return pd.DataFrame(response.json())

# Get Apple's balance sheet
balance = get_balance_sheet("AAPL", "annual", 3)

print("✅ Balance Sheet retrieved!")
print(f"\n📊 Apple Balance Sheet ($ millions):")
key_cols = ["date", "totalAssets", "totalLiabilities", "totalStockholdersEquity"]
display_df = balance[key_cols].copy()
for col in ["totalAssets", "totalLiabilities", "totalStockholdersEquity"]:
    display_df[col] = (display_df[col] / 1e6).round(0).astype(int)
print(display_df.to_string(index=False))

✅ Balance Sheet retrieved!

📊 Apple Balance Sheet ($ millions):
      date  totalAssets  totalLiabilities  totalStockholdersEquity
2025-09-27       359241            285508                    73733
2024-09-28       364980            308030                    56950
2023-09-30       352583            290437                    62146


## Step 8: Get Key Financial Ratios

In [6]:
def get_financial_ratios(ticker: str) -> dict:
    """Get key financial ratios (TTM)."""
    url = f"{BASE_URL}/ratios-ttm/{ticker}"
    params = {"apikey": os.environ["FMP_API_KEY"]}
    response = requests.get(url, params=params)
    data = response.json()
    return data[0] if data else {}

# Get Apple's ratios
ratios = get_financial_ratios("AAPL")

print("✅ Financial Ratios (TTM):")
print(f"\n📊 Valuation:")
print(f"   P/E Ratio: {ratios.get('peRatioTTM', 'N/A'):.2f}")
print(f"   P/B Ratio: {ratios.get('priceToBookRatioTTM', 'N/A'):.2f}")
print(f"   P/S Ratio: {ratios.get('priceToSalesRatioTTM', 'N/A'):.2f}")
print(f"   EV/EBITDA: {ratios.get('enterpriseValueMultipleTTM', 'N/A'):.2f}")

print(f"\n📊 Profitability:")
print(f"   Gross Margin: {ratios.get('grossProfitMarginTTM', 0)*100:.1f}%")
print(f"   Operating Margin: {ratios.get('operatingProfitMarginTTM', 0)*100:.1f}%")
print(f"   Net Margin: {ratios.get('netProfitMarginTTM', 0)*100:.1f}%")
print(f"   ROE: {ratios.get('returnOnEquityTTM', 0)*100:.1f}%")
print(f"   ROA: {ratios.get('returnOnAssetsTTM', 0)*100:.1f}%")

✅ Financial Ratios (TTM):

📊 Valuation:
   P/E Ratio: 33.23
   P/B Ratio: 50.48
   P/S Ratio: 8.84
   EV/EBITDA: 25.90

📊 Profitability:
   Gross Margin: 46.9%
   Operating Margin: 32.0%
   Net Margin: 26.9%
   ROE: 164.0%
   ROA: 31.2%


## Step 9: Get DCF Valuation

In [7]:
def get_dcf_value(ticker: str) -> dict:
    """Get DCF (Discounted Cash Flow) valuation."""
    url = f"{BASE_URL}/discounted-cash-flow/{ticker}"
    params = {"apikey": os.environ["FMP_API_KEY"]}
    response = requests.get(url, params=params)
    data = response.json()
    return data[0] if data else {}

# Get Apple's DCF
dcf = get_dcf_value("AAPL")

if dcf:
    print("✅ DCF Valuation:")
    print(f"   Current Price: ${dcf.get('Stock Price', 'N/A')}")
    print(f"   DCF Value: ${dcf.get('dcf', 'N/A'):.2f}")

    current = dcf.get('Stock Price', 0)
    intrinsic = dcf.get('dcf', 0)
    if current and intrinsic:
        upside = ((intrinsic - current) / current) * 100
        status = "UNDERVALUED" if upside > 0 else "OVERVALUED"
        print(f"   Potential: {upside:+.1f}% ({status})")

✅ DCF Valuation:
   Current Price: $247.65
   DCF Value: $147.46
   Potential: -40.5% (OVERVALUED)


## Step 10: Create Tools for smolagents

In [8]:
!pip install -q smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 3.7 MB/s eta 0:00:00


In [9]:
from smolagents import tool

@tool
def get_financial_metrics(ticker: str) -> dict:
    """Get key financial metrics and ratios from Financial Modeling Prep.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')

    Returns:
        Dictionary with valuation ratios, profitability metrics, and DCF value
    """
    import requests
    import os

    BASE_URL = "https://financialmodelingprep.com/api/v3"
    api_key = os.environ["FMP_API_KEY"]

    # Get ratios
    ratios_url = f"{BASE_URL}/ratios-ttm/{ticker}"
    ratios_resp = requests.get(ratios_url, params={"apikey": api_key})
    ratios = ratios_resp.json()[0] if ratios_resp.json() else {}

    # Get profile for price
    profile_url = f"{BASE_URL}/profile/{ticker}"
    profile_resp = requests.get(profile_url, params={"apikey": api_key})
    profile = profile_resp.json()[0] if profile_resp.json() else {}

    return {
        "ticker": ticker.upper(),
        "name": profile.get("companyName"),
        "price": profile.get("price"),
        "pe_ratio": round(ratios.get("peRatioTTM", 0), 2),
        "pb_ratio": round(ratios.get("priceToBookRatioTTM", 0), 2),
        "roe": round(ratios.get("returnOnEquityTTM", 0) * 100, 1),
        "net_margin": round(ratios.get("netProfitMarginTTM", 0) * 100, 1),
        "debt_to_equity": round(ratios.get("debtEquityRatioTTM", 0), 2),
        "source": "Financial Modeling Prep"
    }

@tool
def get_income_statement_summary(ticker: str) -> dict:
    """Get income statement summary from Financial Modeling Prep.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT')

    Returns:
        Dictionary with recent revenue, profit, and growth metrics
    """
    import requests
    import os

    BASE_URL = "https://financialmodelingprep.com/api/v3"
    api_key = os.environ["FMP_API_KEY"]

    url = f"{BASE_URL}/income-statement/{ticker}"
    response = requests.get(url, params={"apikey": api_key, "limit": 2})
    data = response.json()

    if len(data) >= 2:
        current = data[0]
        previous = data[1]

        revenue_growth = ((current["revenue"] - previous["revenue"]) / previous["revenue"]) * 100
        profit_growth = ((current["netIncome"] - previous["netIncome"]) / previous["netIncome"]) * 100

        return {
            "ticker": ticker.upper(),
            "fiscal_year": current["date"][:4],
            "revenue": current["revenue"],
            "gross_profit": current["grossProfit"],
            "operating_income": current["operatingIncome"],
            "net_income": current["netIncome"],
            "revenue_growth": round(revenue_growth, 1),
            "profit_growth": round(profit_growth, 1),
            "source": "Financial Modeling Prep"
        }

    return {"error": "Insufficient data", "ticker": ticker}

# Test the tools
print("Tool 1 - Financial Metrics:")
print(get_financial_metrics("MSFT"))

print("\nTool 2 - Income Statement:")
print(get_income_statement_summary("MSFT"))

Tool 1 - Financial Metrics:
{'ticker': 'MSFT', 'name': 'Microsoft Corporation', 'price': 445.7, 'pe_ratio': 31.57, 'pb_ratio': 9.12, 'roe': 31.5, 'net_margin': 35.7, 'debt_to_equity': 0.17, 'source': 'Financial Modeling Prep'}

Tool 2 - Income Statement:
{'ticker': 'MSFT', 'fiscal_year': '2025', 'revenue': 281724000000, 'gross_profit': 193893000000, 'operating_income': 128528000000, 'net_income': 101832000000, 'revenue_growth': 14.9, 'profit_growth': 15.5, 'source': 'Financial Modeling Prep'}


## ✅ Setup Complete!

You're ready to use Financial Modeling Prep in the course notebooks.

**Next steps:**
- Save your API key securely (password manager)
- Consider Starter tier ($14/mo) for production use
- Explore SEC filings for detailed company analysis
- Continue to the course Module 1 notebook

**Best for:** Value investing, fundamental analysis, financial modeling.